
# Corrigé Projet Maison 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
GEO = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")


# A COMPLETER avec les colonnes
# - lat, lon : latitude et longitude des communes
# - cp_ville : Code Postal + " " + "Commune"
GEO = (GEO.assign(**GEO.geo_point_2d.str.extract("(?P<lat>.*), (?P<lon>.*)").astype(float))
       .assign(cp_ville=lambda df: df["Code Postal"] + ' ' + df["Commune"])
       .sort_index()
      )

**Partie A**

- Compléter le chargement des données en ajoutant au dataframe `GEO`
    - les colonnes "lat" et "lon" avec la latitude et la longitude des communes
    - une colonne "cp_ville" avec le Code Postal + un espace + et le nom de la Commune
- Ecrire une fonction `search_city(lat, lon)` qui retourne le "cp_ville" de la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ecrire une fonction `dms2dec(deg, min, sec)` qui convertit les degrés, minutes, secondes en valeur numérique pour pouvoir utiliser la fonction précédente avec un GPS.

In [4]:
# fonction recherche de ville
def search_city(lat, long):
    dist2 = ((GEO['lat']-lat)**2 + (GEO['lon']-long)**2)
    return GEO.loc[dist2.idxmin(), "cp_ville"]

In [ ]:
# remarques
# 1)
# boucles for pour calculer le min, pour collecter les résultats du parsing json, etc.
# il faut éviter les boucles et utiliser les fonctions vectorielles de numpy et pandas
# plus efficace, plus concis, pas de bug...
# 2)
# s.loc[s.idxmin()] vs s.loc[s==s.min()]
# 3)
# utiliser la distance euclidienne (approximation), éviter la distance de Manhattan...

In [5]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = np.random.uniform(a, b)
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
lon = np.random.uniform(a, b)

search_city(lat, lon)

'71330 BOSJEAN'

In [6]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [7]:
# à partir de coordonnées GPS précises
search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'91120 PALAISEAU'

**Partie B**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python **json** pour parser les valeurs de la colonne "geo_shape" et mettre le résultat (`Series`) dans la variable `GEO_SHAPE`.
- Ecrire une fonction `get_types()` qui retourne le décompte (`value_counts()`) des valeurs accédées avec la clé "type".
- Ecrire une fonction `get_coordinates_len()` qui retourne le décompte (`value_counts()`) des longueurs des listes accédées avec la clé "coordinates".
- Ecrire une fonction `get_most_complex_city()` qui retourne la commune est constituée du plus grand nombre de polygones ?
- Ecrire une fonction `get_nb_cities_2_polygons()` qui retourne  le nombre de villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- **Facultatif :**
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : on pourra installer la librairie **shapely**, utiliser la classe Polygon de **shapely.geometry**  et la méthode `contains()`. Sur Windows **shapely** peut nécessiter d'installer manuellement la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [8]:
# GEO_SHAPE

# La variable GEO_SHAPE doit contenir une Serie
# correspondant aux valeurs de la colonne "geo_shape" parsées avec la librairie json
import json

GEO_SHAPE = GEO['geo_shape'].apply(json.loads)
GEO_SHAPE

Code INSEE
01001    {'type': 'Polygon', 'coordinates': [[[4.926273...
01002    {'type': 'Polygon', 'coordinates': [[[5.430089...
01004    {'type': 'Polygon', 'coordinates': [[[5.386190...
01005    {'type': 'Polygon', 'coordinates': [[[4.895580...
01006    {'type': 'Polygon', 'coordinates': [[[5.614854...
01007    {'type': 'Polygon', 'coordinates': [[[5.413533...
01008    {'type': 'Polygon', 'coordinates': [[[5.321986...
01009    {'type': 'Polygon', 'coordinates': [[[5.656393...
                               ...                        
97610    {'type': 'Polygon', 'coordinates': [[[45.23692...
97611    {'type': 'Polygon', 'coordinates': [[[45.19781...
97612    {'type': 'MultiPolygon', 'coordinates': [[[[45...
97613    {'type': 'Polygon', 'coordinates': [[[45.10168...
97614    {'type': 'Polygon', 'coordinates': [[[45.15401...
97615    {'type': 'Polygon', 'coordinates': [[[45.29645...
97616    {'type': 'Polygon', 'coordinates': [[[45.13226...
97617    {'type': 'Polygon', 'coordinates': [

In [9]:
# value_counts des valeurs "type"
def get_types():
    return GEO_SHAPE.apply(lambda x: x['type']).value_counts()

In [10]:
get_types()

Polygon         36670
MultiPolygon       72
Name: geo_shape, dtype: int64

In [11]:
# value_counts des longueurs de "coordinates"
def get_coordinates_len():
    return GEO_SHAPE.apply(lambda x: x['coordinates']).apply(len).value_counts()

In [12]:
get_coordinates_len()

1    36660
2       80
3        1
4        1
Name: geo_shape, dtype: int64

In [13]:
# commune constituée du plus grand nombre de polygones
def get_most_complex_city():
    return GEO.loc[GEO_SHAPE.apply(lambda x: x['coordinates']).apply(len).idxmax(), "cp_ville"]

In [14]:
get_most_complex_city()

'83400 HYERES'

In [15]:
# nombre de villes de type "Polygon" dont la longueur des listes accédées avec la clé "coordinates" vaut 2
def get_nb_cities_2_polygons():
    return (GEO_SHAPE.apply(lambda x: x['type']=='Polygon') &
            (GEO_SHAPE.apply(lambda x: x['coordinates']).apply(len)==2)).sum()

In [16]:
get_nb_cities_2_polygons()

10

In [ ]:
# remarque
# il faut rester dans pandas pour effectuer des sélections
# notamment en utilisant les opérateurs logiques (cf. SQL)
# penser également à additionner les booléens et aux méthodes all() et any()

In [20]:
# vérification que ce sont des enclaves
# universalité de numpy

from shapely.geometry import Polygon

def check_enclaves():

    def test_enclave(coordinates):
        if len(coordinates)!=2:
            return False
        p1 = Polygon(coordinates[0])
        p2 = Polygon(coordinates[1])
        return p1.contains(p2)

    selection = GEO_SHAPE.loc[GEO_SHAPE.apply(lambda x: x['type']=='Polygon') &
                              (GEO_SHAPE.apply(lambda x: x['coordinates']).apply(len)==2)]

    return selection.apply(lambda x: test_enclave(x['coordinates'])).all()

In [21]:
check_enclaves()

True

In [22]:
# tests
import unittest

class Session2Test(unittest.TestCase):
    
    def test_partie_A1(self):
        # on applique la fonction cherche_ville() à une coordonnée tirée au hasard
        np.random.seed(0)
        a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
        lat = np.random.uniform(a, b)
        a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
        lon = np.random.uniform(a, b)

        cp_ville = search_city(lat, lon)
        self.assertEqual(cp_ville, "71330 BOSJEAN")
        
    def test_partie_A2(self):
        # à partir de coordonnées GPS précises
        cp_ville = search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))
        self.assertEqual(cp_ville, "91120 PALAISEAU")
        
    def test_partie_B1(self):
        # check types
        dico = get_types()
        self.assertEqual(dico["Polygon"], 36670)
        self.assertEqual(dico["MultiPolygon"], 72)
        
    def test_partie_B2(self):
        # check coordinates len
        dico = get_coordinates_len()
        self.assertEqual(dico[1], 36660)
        self.assertEqual(dico[2], 80)
       
    def test_partie_B3(self):
        # check most complex city
        cp_ville = get_most_complex_city()
        self.assertEqual(cp_ville, "83400 HYERES")
        
    def test_partie_B4(self):
        # check nb cities 2 polygons
        nb = get_nb_cities_2_polygons()
        self.assertEqual(nb, 10)

In [23]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session2Test)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

test_partie_A1 (__main__.Session2Test) ... ok
test_partie_A2 (__main__.Session2Test) ... ok
test_partie_B1 (__main__.Session2Test) ... ok
test_partie_B2 (__main__.Session2Test) ... ok
test_partie_B3 (__main__.Session2Test) ... ok
test_partie_B4 (__main__.Session2Test) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.287s

OK
